In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np

# Tạo dữ liệu tổng hợp
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=2, random_state=42)

# Chuyển đổi sang DataFrame để dễ nhìn
df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
df['target'] = y


x_train, x_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.3, random_state=42)

x_train = x_train.values
y_train = y_train.values
x_test = x_test.values
y_test = y_test.values

In [ ]:
def loss_function(y, y_hat):
    """Tính toán hàm mất mát (Binary Cross-Entropy)"""
    # Thêm một giá trị epsilon nhỏ để tránh log(0)
    epsilon = 1e-15
    y_hat = np.clip(y_hat, epsilon, 1 - epsilon)
    return -np.mean(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))

def sigmoid_function(x):
    """Hàm Sigmoid"""
    return 1 / (1 + np.exp(-x))

def predict(x, weight, bias):
    """Tính giá trị dự đoán y = wx + b"""
    weight = np.array(weight)
    return sigmoid_function(np.dot(x, weight) + bias)

def compute_gradients(x, y, weight, bias):
    """Tính gradient cho weight và bias"""
    predictions = predict(x, weight, bias)
    errors = predictions - y # Tính sai số
    dw = np.dot(x.T, errors) / len(y) # Gradient cho trọng số
    db = np.mean(errors) # Gradient cho độ lệch
    return dw, db

def train_logistic_regression(x, y, learning_rate=0.1, epochs=1000):
    """
    Huấn luyện mô hình hồi quy logistic sử dụng gradient descent
    Args:
        x: numpy array, biến độc lập
        y: numpy array, biến phụ thuộc
        learning_rate: tốc độ học
        epochs: số vòng lặp huấn luyện
    Returns:
        weight, bias: tham số mô hình đã huấn luyện
    """

    weight = np.ones(x.shape[1])
    bias = 0.0

    # Kiểm tra đầu vào
    if len(x) != len(y):
        raise ValueError("Kích thước của x và y phải bằng nhau")
    if len(x) == 0:
        raise ValueError("Dữ liệu đầu vào rỗng")

    # Huấn luyện mô hình
    for epoch in range(epochs):
        # Tính gradient
        dw, db = compute_gradients(x, y, weight, bias)

        # Cập nhật tham số
        weight -= learning_rate * dw
        bias -= learning_rate * db

        # In loss sau mỗi vài epoch để theo dõi
        if (epoch+1) % 100 == 0:
            loss = loss_function(y, predict(x, weight, bias))
            print(f"Epoch {epoch+1}, Loss: {loss:.4f}")


    return weight, bias

try:
    st = time.time()
    final_weight, final_bias = train_logistic_regression(x_train, y_train) # Sử dụng .values để lấy numpy array từ Series
    ed = time.time()
    print(f"\nThời gian huấn luyện: {ed-st:.4f} giây")
    # In trọng số và độ lệch
    print(f"Bias cuối cùng: {final_bias:.4f}")
    print("Weight cuối cùng:")
    print(final_weight)


except Exception as e:
    print(f"Lỗi trong quá trình huấn luyện: {str(e)}")

Epoch 100, Loss: 0.4660
Epoch 200, Loss: 0.4439
Epoch 300, Loss: 0.4421
Epoch 400, Loss: 0.4418
Epoch 500, Loss: 0.4417
Epoch 600, Loss: 0.4417
Epoch 700, Loss: 0.4417
Epoch 800, Loss: 0.4417
Epoch 900, Loss: 0.4417
Epoch 1000, Loss: 0.4417

Thời gian huấn luyện: 0.0411 giây
Bias cuối cùng: 0.6661
Weight cuối cùng:
[ 0.49579574  0.35306445  0.02847362  1.73143567 -0.46153487 -0.0402296
 -0.53350539 -0.11800595  0.02896067  0.61709712]


In [ ]:
correct = 0

logits = predict(x_test, final_weight, final_bias)
predictions = np.round(logits)
loss = loss_function(y_test, logits)
correct += (predictions == y_test).sum()
print(f"Accuracy: {correct/len(y_test)*100}%")
print(f"Loss: {loss}")

Accuracy: 82.0%
Loss: 0.3842083716720332
